In [6]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "C:/Users/ASUS/Downloads/project/cctv.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            print("track",track)
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 0, 250), thickness=4)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 4 persons, 2 cars, 46.9ms
Speed: 0.0ms preprocess, 46.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
track [(969.7706298828125, 359.6568603515625)]
track [(650.86279296875, 448.29962158203125)]
track [(442.30426025390625, 142.27877807617188)]
track [(245.17501831054688, 124.91952514648438)]
track [(201.62054443359375, 128.41403198242188)]
track [(796.0023193359375, 220.4819793701172)]

0: 384x640 4 persons, 2 cars, 27.7ms
Speed: 3.4ms preprocess, 27.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
track [(969.7706298828125, 359.6568603515625), (969.802490234375, 359.5895690917969)]
track [(650.86279296875, 448.29962158203125), (647.6820068359375, 445.6227722167969)]
track [(442.30426025390625, 142.27877807617188), (442.2752685546875, 142.24032592773438)]
track [(245.17501831054688, 124.91952514648438), (244.69635009765625, 123.26832580566406)]
track [(201.62054443359375, 128.41403198242188), (201.6929931640625, 127.00956726074219)]
